In [1]:
######################################################################################################################
###### Script for reordering fastMASST results, merging with ReDU and getting information on datasets related to #####
###### humans and rodents to build Figure 2 a and b #################################################################
######################################################################################################################


# Getting the fastMASST combined results

In [ ]:
# First, we need to open and concatenate vertically all the fastMASST results. For this, we need to also add
# important information about the file (that are in the fileame itself), such as the BA class (mono, di, etc),
# and the scan number.

In [ ]:
#import the fastmasst summary results (filenames of all the files generated in the workflow)
#For more information about this workflow, check https://github.com/robinschmid/microbe_masst

result_summary = pd.read_csv('MASST_summary_all.txt', sep='\t')

In [ ]:
#keep only rows that contain 'matches' in the first row, to focus separatelly them
result_summary_fastmasst = result_summary[result_summary['filename'].str.contains('_matches.tsv')]

In [ ]:
#now we create a loop to read all of these as dataframes, then we create a temporary file, and concatenate them all 

fastmasst_all = pd.DataFrame()
current = 1

for i in range(len(result_summary_fastmasst)):
    filename = result_summary_fastmasst.iloc[i]['filename']
    df_temp = pd.read_csv('/Users/helenarusso/Documents/Pesquisa/UCSD Post-doc/Bile acids mining MassQL/microbeMASST/microbeMASST_bile_acids_all_results.nosync/Non_Refined_OK/'+filename, sep='\t')
    df_temp['Bile acid'] = result_summary_fastmasst.iloc[i]['Bile acid']
    df_temp['Scan'] = result_summary_fastmasst.iloc[i]['Scan']
    fastmasst_all = pd.concat([fastmasst_all, df_temp], axis=0)
    print('Opening {} of {}'.format(current, len(result_summary_fastmasst)))
    current += 1


In [ ]:
fastmasst_all.to_csv('/Users/helenarusso/Downloads/fastmasst_all_non_refined.tsv', sep='\t')

# Import all MassQL results to get the precmz and calculate the delta masses

In [ ]:
#import all MassQL data. Cluster index is the scan number of the mgf files sent to for MASST analyses.

df_non = pd.read_csv('Nonhydroxy_MSCluster_MCS2_non_refined/Nonhydroxy_75a705dd-view_all_clusters_withID_beta-main.tsv', sep='\t')
df_mono = pd.read_csv('Monohydroxy_MSCluster_MCS2_non_refined/Monohydroxy_88b52c90-view_all_clusters_withID_beta-main.tsv', sep='\t')
df_di = pd.read_csv('Dihydroxy_MSCluster_MCS2_non_refined/Dihydroxy_2frag_11424cb4-view_all_clusters_withID_beta-main.tsv', sep='\t')
df_tri = pd.read_csv('Trihydroxy_MSCluster_MCS2_non_refined/Trihydroxy_85b060ec-view_all_clusters_withID_beta-main.tsv', sep='\t')
df_tetra = pd.read_csv('Tetrahydroxy_MSCluster_MCS2_non_refined/Tetrahydroxy_b51812c1-view_all_clusters_withID_beta-main.tsv', sep='\t')
df_penta = pd.read_csv('Pentahydroxy_MSCluster_MCS2_non_refined/Pentahydroxy_b8404e55-view_all_clusters_withID_beta-main.tsv', sep='\t')


In [ ]:
#calculating values for a new column to be populated by delta masses. In this case, we first subtract the
#precmz column by a proton (assuming that it is a M+H adduct), and then subtract by the uncharged bile acid.

df_non["delta_mass"] = (df_non['precursor mass'] - 361.3101)
df_mono["delta_mass"] = (df_mono['precursor mass'] - 377.3050)
df_di["delta_mass"] = (df_di['precursor mass'] - 393.2999)
df_tri["delta_mass"] = (df_tri['precursor mass'] - 409.2949)
df_tetra["delta_mass"] = (df_tetra['precursor mass'] - 425.2898)
df_penta["delta_mass"] = (df_penta['precursor mass'] - 441.2847)

In [ ]:
#Calculating absolute numbers (multiply (-1) the negative values)

df_non["delta_mass"] = df_non["delta_mass"].abs()
df_mono["delta_mass"] = df_mono["delta_mass"].abs()
df_di["delta_mass"] = df_di["delta_mass"].abs()
df_tri["delta_mass"] = df_tri["delta_mass"].abs()
df_tetra["delta_mass"] = df_tetra["delta_mass"].abs()
df_penta["delta_mass"] = df_penta["delta_mass"].abs()

In [ ]:
#Now we concatenate all the tables vertically

df_all = pd.concat([df_non, df_mono, df_di, df_tri, df_tetra, df_penta], axis=0)
df_all = df_all.reset_index()

In [ ]:
#For us to compare properly, we can have the 'Bile_acid_GNPSjob' + 'cluster index' to merge tables:

df_all['Bile_scan'] = df_all['Bile_acid'].astype(str).str.cat(df_all[['cluster index']].astype(str), sep='_')

In [ ]:
#and also do it for the fastMASST table:
fastmasst_all_ok = fastmasst_all.copy()
fastmasst_all_ok['Bile_scan'] = fastmasst_all['Bile acid'].astype(str).str.cat(fastmasst_all[['Scan']].astype(str),
                                                                               sep='_')

In [ ]:
#now we merge the two big tables by the Bile_scan column :D
merged_tables_all = pd.merge(fastmasst_all_ok, df_all[['Bile_scan', 'precursor mass', 'delta_mass']],
                             on = "Bile_scan", how = "left")

#and reset index
merged_tables_all = merged_tables_all.reset_index()
#Remove the 'index' column as it can get confusing in the middle
merged_tables_all = merged_tables_all.drop('index', axis=1)

In [ ]:
#now for us to merge with ReDU, we need to have a column with dataset/filename
merged_tables_all["dataset"] = merged_tables_all["USI"].str.split(":").str[1]
merged_tables_all["filename"] = merged_tables_all["USI"].str.split(":").str[2]
merged_tables_all['filepath'] = merged_tables_all['dataset'].astype(str).str.cat(merged_tables_all[['filename']].astype(str),
                                                                                 sep='/')

# ReDU merging

In [ ]:
#Import ReDU Database (obtained from https://redu.ucsd.edu/ --> Download Database)
df_redu = pd.read_csv('/Users/helenarusso/Documents/Pesquisa/UCSD Post-doc/Bile acids mining MassQL/MassQL ok - Histograms.nosync/All/ReDU/all_sampleinformation.tsv', sep='\t')


In [ ]:
#remove 'f.' from filename column, create new column with the dataset and last element of the filename column, and 
#remove the extension of the file (.mzML/.mzML) to match the column we have in the other table.
df_redu['filename'] = df_redu['filename'].str.replace('f.', '')
df_redu['filename_2'] = df_redu['filename'].str.split('/').str[-1]
df_redu['filename_2'] = df_redu['filename_2'].str.replace('.mzML', '')
df_redu['filename_2'] = df_redu['filename_2'].str.replace('.mzXML', '')
df_redu['filepath'] = df_redu['ATTRIBUTE_DatasetAccession'].astype(str).str.cat(df_redu[['filename_2']].astype(str),
                                                                                sep='/')

In [ ]:
#merge tables from fastmasst and ReDU
merged_tables_all_redu = pd.merge(merged_tables_all, df_redu, on='filepath', how='left')
merged_tables_all_redu.fillna("N/A", inplace = True)

In [ ]:
#drop some columns
merged_tables_all_redu = merged_tables_all_redu.drop('filename_2', axis=1)
merged_tables_all_redu = merged_tables_all_redu.drop('Status', axis=1)
merged_tables_all_redu = merged_tables_all_redu.drop('level_0', axis=1)

In [ ]:
#export table
merged_tables_all_redu.to_csv('/Users/helenarusso/Downloads/fastmasst_all_merged_ReDU_non_refined_final.tsv', sep='\t')


In [ ]:
#Now let's keep only a few columns for clarity

merged_tables_all_redu_columns = merged_tables_all_redu[['Bile acid', 'Cosine', 'Matching Peaks', 'Scan', 'USI', 
                                                         'Bile_scan', 'precursor mass', 'delta_mass', 'dataset',
                                                         'filename_x', 'filepath', 'filename_y', 'BiologicalSex', 
                                                         'HealthStatus', 'LifeStage', 'MassSpectrometer', 
                                                         'NCBITaxonomy', 'UBERONBodyPartName']]

# Organizing the table for the body part figure (Supplementary Figure S1)

In [ ]:
#First step is just keeping the rows that are from datasets related to Homo sapiens

merged_tables_all_homo_sapiens = merged_tables_all_redu_columns.loc[merged_tables_all_redu_columns['NCBITaxonomy'] == '9606|Homo sapiens']


In [ ]:
#check how many unique UBERONBodyPartName there is in the whole data we have
merged_tables_all_homo_sapiens["UBERONBodyPartName"].explode().unique()


In [ ]:
#create a list with the body part names
list_body_homo_sapiens = ['saliva', 'urine', 'skin of trunk', 'blood plasma','skin of manus', 'head or neck skin', 
                          'arm skin', 'anal region', 'oral cavity', 'feces', 'nasal cavity', 'cerebrospinal fluid',
                          'vagina', 'milk', 'brain', 'blood serum', 'not applicable', 'skin of pes', 'skin of body', 
                          'skin of leg', 'axilla skin']

#and add this as rows in the first column of a new dataframe
df_body_part_homo_sapiens = pd.DataFrame(columns=['UBERONBodyPartName'])
df_body_part_homo_sapiens['UBERONBodyPartName'] = np.array(list_body_homo_sapiens)
df_body_part_homo_sapiens

In [ ]:
#now we divide the dataframe into non up to pentahydroxylated bile acids

merged_tables_non_homo_sapiens = merged_tables_all_homo_sapiens.loc[merged_tables_all_homo_sapiens['Bile acid'] == 'Nonhydroxy']
merged_tables_mono_homo_sapiens = merged_tables_all_homo_sapiens.loc[merged_tables_all_homo_sapiens['Bile acid'] == 'Monohydroxy']
merged_tables_di_homo_sapiens = merged_tables_all_homo_sapiens.loc[merged_tables_all_homo_sapiens['Bile acid'] == 'Dihydroxy']
merged_tables_tri_homo_sapiens = merged_tables_all_homo_sapiens.loc[merged_tables_all_homo_sapiens['Bile acid'] == 'Trihydroxy']
merged_tables_tetra_homo_sapiens = merged_tables_all_homo_sapiens.loc[merged_tables_all_homo_sapiens['Bile acid'] == 'Tetrahydroxy']
merged_tables_penta_homo_sapiens = merged_tables_all_homo_sapiens.loc[merged_tables_all_homo_sapiens['Bile acid'] == 'Pentahydroxy']


In [ ]:
#now we create new dataframes with the counts of each time a body part occur. This is the total_deltas column.
#With this, we have the information of how common these deltas are (not necessarily the diversity)
merged_tables_non_homo_sapiens_BodyPartName = merged_tables_non_homo_sapiens['UBERONBodyPartName'].value_counts().rename_axis('UBERONBodyPartName').reset_index(name='Non_total_deltas')
merged_tables_mono_homo_sapiens_BodyPartName = merged_tables_mono_homo_sapiens['UBERONBodyPartName'].value_counts().rename_axis('UBERONBodyPartName').reset_index(name='Mono_total_deltas')
merged_tables_di_homo_sapiens_BodyPartName = merged_tables_di_homo_sapiens['UBERONBodyPartName'].value_counts().rename_axis('UBERONBodyPartName').reset_index(name='Di_total_deltas')
merged_tables_tri_homo_sapiens_BodyPartName = merged_tables_tri_homo_sapiens['UBERONBodyPartName'].value_counts().rename_axis('UBERONBodyPartName').reset_index(name='Tri_total_deltas')
merged_tables_tetra_homo_sapiens_BodyPartName = merged_tables_tetra_homo_sapiens['UBERONBodyPartName'].value_counts().rename_axis('UBERONBodyPartName').reset_index(name='Tetra_total_deltas')
merged_tables_penta_homo_sapiens_BodyPartName = merged_tables_penta_homo_sapiens['UBERONBodyPartName'].value_counts().rename_axis('UBERONBodyPartName').reset_index(name='Penta_total_deltas')


In [ ]:
#then, we merge all these tables (has to be one by one)

df_body_part_homo_sapiens = pd.merge(merged_tables_non_homo_sapiens_BodyPartName,
                              merged_tables_mono_homo_sapiens_BodyPartName, on = "UBERONBodyPartName", how = "outer")
df_body_part_homo_sapiens.fillna("N/A", inplace = True)

df_body_part_homo_sapiens = pd.merge(df_body_part_homo_sapiens,
                              merged_tables_di_homo_sapiens_BodyPartName, on = "UBERONBodyPartName", how = "outer")
df_body_part_homo_sapiens.fillna("N/A", inplace = True)

df_body_part_homo_sapiens = pd.merge(df_body_part_homo_sapiens,
                              merged_tables_tri_homo_sapiens_BodyPartName, on = "UBERONBodyPartName", how = "outer")
df_body_part_homo_sapiens.fillna("N/A", inplace = True)

df_body_part_homo_sapiens = pd.merge(df_body_part_homo_sapiens,
                              merged_tables_tetra_homo_sapiens_BodyPartName, on = "UBERONBodyPartName", how = "outer")
df_body_part_homo_sapiens.fillna("N/A", inplace = True)

df_body_part_homo_sapiens = pd.merge(df_body_part_homo_sapiens,
                              merged_tables_penta_homo_sapiens_BodyPartName, on = "UBERONBodyPartName", how = "outer")
df_body_part_homo_sapiens.fillna("N/A", inplace = True)

In [ ]:
df_body_part_homo_sapiens.to_csv('/Users/helenarusso/Downloads/body_parts_homo_sapiens_total_2decimals_fastmasst_non_refined.tsv', sep='\t')
